In [1]:
# install gensim for doc2vec
!pip install --upgrade gensim

  Using cached gensim-4.0.1.tar.gz (23.1 MB)
Using legacy 'setup.py install' for gensim, since package 'wheel' is not installed.
    Running setup.py install for gensim: started
    Running setup.py install for gensim: finished with status 'done'


You should consider upgrading via the 'e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [103]:
cb_data = pd.read_csv('../../../../data/organization_descriptions.csv', encoding='utf-8-sig')
corepo_orgs = pd.read_csv('corepo_lookup_labelled.csv', index_col=0)

In [104]:
cb_data[cb_data['name']=='FamPay'].iloc[0]['description']

'FamPay is India’s first payments app for teenagers and their families. With FamPay, minors can do UPI, P2P and card payments without the need to set up a bank account. FamPay allows parents to send money to their kids below the age of 18, which they can spend securely anytime, anywhere, under parent’s supervision. FamPay aims at making payments smooth and fun for children.'

In [105]:
corepo_orgs.iloc[0]['description']

'FamPay is a fintech company developing a payment platform for teenagers. ...                    - industries: Technology, fintech, mobile app, payments, platform'

In [76]:
sampled_data = cb_data.sample(n=50000)
data = sampled_data['description']

data = data.astype(str)
data

173560    Media consumption is changing, so should the t...
619725    webexcept IT Services offers a wide array of w...
222898    ID’SIGN TECHNOLOGIES is a tech company capable...
242457    EarBuddies:tm: are the first headphones with 1...
88838     Catho is a technology company that functions a...
                                ...                        
115494    Electron Hut is a gaming company and has creat...
654429    Aqua Dynamo is an incredibly powerful tidal tu...
623748    UTunnel VPN provides On-Premise and Cloud VPN ...
313892    SWAAY is a ground-breaking media and online pu...
414316    TEBETEC AG develops and produces solutions and...
Name: description, Length: 50000, dtype: object

In [77]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

In [78]:
# train and save the model
max_epochs = 100
vec_size = 100
alpha = 0.025

model = Doc2Vec(alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1) #,vector_size=vec_size)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

In [123]:
model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
# text = "FamPay is a fintech company developing a payment platform for teenagers. ...                    - industries: Technology, fintech, mobile app, payments, platform".lower()
text = ""
test_data = word_tokenize(text)
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.dv.most_similar([v1])
print(similar_doc)
print(data.iloc[7194], data.iloc[6549], data.iloc[8831], data.iloc[39365])

# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
# print(model.docvecs['1'])

V1_infer [-1.4772728e-03 -1.4114880e-03  2.6487100e-03  4.4517801e-03
 -9.8124240e-04 -4.1154418e-03  4.9785208e-03  9.2476013e-04
  2.7315307e-03  1.9042493e-05 -4.1804002e-03 -1.7291760e-03
  3.7102698e-05 -1.4509830e-03 -4.3476964e-03 -4.2421431e-03
 -1.6554976e-03 -4.1985130e-03 -4.4868528e-04 -3.3265210e-03
 -1.7995232e-03 -2.2351919e-03 -1.0433972e-04  4.9278913e-03
 -4.9101142e-04 -8.7959645e-04  3.3418750e-03 -4.2607598e-03
 -3.4603751e-03 -1.5155703e-04 -8.4123012e-06 -2.6194035e-04
 -2.3491404e-03  4.0471931e-03 -4.1657449e-03 -4.6190289e-03
 -2.6746243e-03  3.0290305e-03 -4.1952999e-03 -4.6291845e-03
 -2.6893592e-03  1.3656878e-03  2.8337550e-03 -3.9617871e-03
 -4.9232789e-03 -2.0322630e-03  1.3946575e-03  3.2662374e-03
  4.9045808e-03  7.3369680e-04  3.4060620e-03  2.3543751e-03
  1.6464847e-03 -2.0839826e-03  1.9010317e-03  4.7629951e-03
 -2.0503358e-03 -7.0902973e-04 -2.7636541e-03 -2.0096251e-03
  3.5361361e-03 -4.5243948e-04  2.9994762e-03  1.7675608e-03
 -3.6750145e-03

In [82]:
# load model
model= Doc2Vec.load("d2v.model")

In [106]:
# create new column to store vectors
corepo_orgs['description_vector'] = None
# force convert description column to string
corepo_orgs['description'] = corepo_orgs['description'].astype(str)

In [109]:
for i in range(0, len(corepo_orgs)):
    print(i, end=' ')
    text = corepo_orgs.iloc[i]['description']
    text = ' '.join(text.split()).lower()
    vector = model.infer_vector(test_data)
    corepo_orgs['description_vector'].iloc[i] = vector

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123

e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373

2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024 2025 2026 2027 2028 2029 2030 2031 2032 2033 2034 2035 2036 2037 2038 2039 2040 2041 2042 2043 2044 2045 2046 2047 2048 2049 2050 2051 2052 2053 2054 2055 2056 2057 2058 2059 2060 2061 2062 2063 2064 2065 2066 2067 2068 2069 2070 2071 2072 2073 2074 2075 2076 2077 2078 2079 2080 2081 2082 2083 2084 2085 2086 2087 2088 2089 2090 2091 2092 2093 2094 2095 2096 2097 2098 2099 2100 2101 2102 2103 2104 2105 2106 2107 2108 2109 2110 2111 2112 2113 2114 2115 2116 2117 2118 2119 2120 2121 2122 2123 2124 2125 2126 2127 2128 2129 2130 2131 2132 2133 2134 2135 2136 2137 2138 2139 2140 2141 2142 2143 2144 2145 2146 2147 2148 2149 2150 2151 2152 2153 2154 2155 2156 2157 2158 2159 2160 2161 2162 2163 2164 2165 2166 2167 2168 2169 2170 2171 2172 2173 2174 2175 2176 2177 2178 2179 2180 2181 2182 2183 2184 2185 2186 2187 2188 2189 2190 2191 2192 2193 2194 2195 2196 2197 2198 2199 2200 2201 2202 2203 

 3669 3670 3671 3672 3673 3674 3675 3676 3677 3678 3679 3680 3681 3682 3683 3684 3685 3686 3687 3688 3689 3690 3691 3692 3693 3694 3695 3696 3697 3698 3699 3700 3701 3702 3703 3704 3705 3706 3707 3708 3709 3710 3711 3712 3713 3714 3715 3716 3717 3718 3719 3720 3721 3722 3723 3724 3725 3726 3727 3728 3729 3730 3731 3732 3733 3734 3735 3736 3737 3738 3739 3740 3741 3742 3743 3744 3745 3746 3747 3748 3749 3750 3751 3752 3753 3754 3755 3756 3757 3758 3759 3760 3761 3762 3763 3764 3765 3766 3767 3768 3769 3770 3771 3772 3773 3774 3775 3776 3777 3778 3779 3780 3781 3782 3783 3784 3785 3786 3787 3788 3789 3790 3791 3792 3793 3794 3795 3796 3797 3798 3799 3800 3801 3802 3803 3804 3805 3806 3807 3808 3809 3810 3811 3812 3813 3814 3815 3816 3817 3818 3819 3820 3821 3822 3823 3824 3825 3826 3827 3828 3829 3830 3831 3832 3833 3834 3835 3836 3837 3838 3839 3840 3841 3842 3843 3844 3845 3846 3847 3848 3849 3850 3851 3852 3853 3854 3855 3856 3857 3858 3859 3860 3861 3862 3863 3864 3865 3866 3867 3868

5424 5425 5426 5427 5428 5429 5430 5431 5432 5433 5434 5435 5436 5437 5438 5439 5440 5441 5442 5443 5444 5445 5446 5447 5448 5449 5450 5451 5452 5453 5454 5455 5456 5457 5458 5459 5460 5461 5462 5463 5464 5465 5466 5467 5468 5469 5470 5471 5472 5473 5474 5475 5476 5477 5478 5479 5480 5481 5482 5483 5484 5485 5486 5487 5488 5489 5490 5491 5492 5493 5494 5495 5496 5497 5498 5499 5500 5501 5502 5503 5504 5505 5506 5507 5508 5509 5510 5511 5512 5513 5514 5515 5516 5517 5518 5519 5520 5521 5522 5523 5524 5525 5526 5527 5528 5529 5530 5531 5532 5533 5534 5535 5536 5537 5538 5539 5540 5541 5542 5543 5544 5545 5546 5547 5548 5549 5550 5551 5552 5553 5554 5555 5556 5557 5558 5559 5560 5561 5562 5563 5564 5565 5566 5567 5568 5569 5570 5571 5572 5573 5574 5575 5576 5577 5578 5579 5580 5581 5582 5583 5584 5585 5586 5587 5588 5589 5590 5591 

In [110]:
corepo_orgs

,article_id,search_keyword,name,description,domain,founding_year,funding,employee_count,industry,location,city,country,linkedin,country_code,funding_numeric,linkedin_identifier,last_funding_type,employee_count_original,description_vector
0,0,FamPay,FamPay,FamPay is a fintech company developing a payme...,https://fampay.in,2019.0,4.7 million,51-500,"Technology, fintech, mobile app, payments, pla...",NaN,NaN,NaN,https://www.linkedin.com/company/fampay/,unknown,4700000.0,fampay,seed,51 - 200,"[-0.22809392, -0.057925172, 0.11451828, 0.5015..."
1,1,Ontario Teachers Pension Plan,Ontario Teachers' Pension Plan,Ontario Teachers&s; Pension Plan invests and a...,https://otpp.com,1989.0,NaN,1001-5000,"Financial Services, Investment, financial mana...","Toronto, Canada",Toronto,Canada,http://www.linkedin.com/company/ontario-teache...,CAN,unknown,ontario-teachers&s;-pension-plan,NaN,"1,001 - 5,000","[-0.17474109, 0.022643274, 0.087769695, 0.4400..."
2,1,Ontario Teachers Pension Plan,Cadillac Fairview Corporation Limited,Cadillac Fairview is one of the largest owners...,https://cadillacfairview.com,1974.0,NaN,1001-5000,Real Estate,"Toronto, Canada",Toronto,Canada,https://www.linkedin.com/company/cadillac-fair...,CAN,unknown,cadillac-fairview,NaN,"1,001 - 5,000","[-0.24376787, 0.009943284, 0.09498452, 0.52470..."
3,1,Ontario Teachers Pension Plan,Canadian Tire Corporation Limited,"Get anything from auto parts to home décor, ou...",https://canadiantire.ca,1922.0,NaN,10000+,NaN,"Toronto, Canada",Toronto,Canada,https://www.linkedin.com/company/canadian-tire/,CAN,unknown,canadian-tire,NaN,"10,000 or more","[-0.24015686, 0.01945192, 0.048324544, 0.45081..."
4,1,Ontario Teachers Pension Plan,Maple Leaf Sports & Entertainment,Maple Leaf Sports & Entertainment (MLSE) is a ...,https://mlse.com,1998.0,NaN,1001-5000,"Media & Entertainment, event management, games","Toronto, Canada",Toronto,Canada,https://www.linkedin.com/company/maple-leaf-sp...,CAN,unknown,maple-leaf-sports-&-entertainment,private_equity,"1,001 - 5,000","[-0.27525356, 0.022101998, 0.006976576, 0.4300..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9652,499,Copenhagen,Fuze,If you're searching for a unified enterprise c...,https://fuze.com,2006.0,481.0 million,501-1000,"Technology, cloud, enterprise software, messag...","Boston, United states",Boston,United states,https://www.linkedin.com/company/fuze-inc/,USA,481000000.0,fuze-inc,series_f,"501 - 1,000","[-0.28085235, -0.080637336, 0.05297814, 0.4747..."
9653,499,Copenhagen,Siteimprove,One suite for improving your website through S...,https://siteimprove.com,2003.0,55.0 million,501-1000,"Technology, platform, seo","Minneapolis, United states",Minneapolis,United states,http://www.linkedin.com/company/siteimprove,USA,55000000.0,siteimprove,series_unknown,"501 - 1,000","[-0.32234484, 0.02471948, 0.16528614, 0.453018..."
9655,499,Copenhagen,citizenM Hotel,citizenM hotels offer affordable luxury in som...,https://citizenm.com,2005.0,840.3 million,501-1000,"Hotels, Motels, & Resorts","Voorschoten, Netherlands",Voorschoten,Netherlands,https://www.linkedin.com/company/citizenm,NLD,840300000.0,citizenm,private_equity,"501 - 1,000","[-0.3720409, -0.049984973, 0.05586882, 0.56369..."
9656,499,Copenhagen,Issuu,"Youtube for Magazines , Love it. Clip it. Shar...",https://issuu.com,2007.0,20.3 million,51-500,"Media & Entertainment, magazine, news, newspap...","Palo alto, United states",Palo alto,United states,https://www.linkedin.com/company/issuu,USA,20300000.0,issuu,series_b,51 - 200,"[-0.25918737, 0.018739216, 0.087273434, 0.4497..."


In [119]:
max_epochs = 100
vec_size = 100
alpha = 0.025

model2 = Doc2Vec(alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
#                 compute_loss=True,
                dm =1) #,vector_size=vec_size)
  
model2.build_vocab(tagged_data)



In [120]:
%%time
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model2.train(tagged_data,
                total_examples=model2.corpus_count,
                epochs=model2.epochs, compute_loss=True)
    print(model2.get_latest_training_loss())
    # decrease the learning rate
    model2.alpha -= 0.0002
    # fix the learning rate, no decay
    model2.min_alpha = model2.alpha

iteration 0
0.0
iteration 1


KeyboardInterrupt: 